# Prepare and visualize data
## Import libraries




In [2]:
import pandas as pd
import plotly.graph_objects as go

## Import data

### Kieler Woche

In [202]:
import datetime

df_kiwo = pd.read_csv('../data/kiwo.csv')#, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_kiwo.head()
#print(type(df_kiwo))

,Datum,KielerWoche
0,2012-06-16,1
1,2012-06-17,1
2,2012-06-18,1
3,2012-06-19,1
4,2012-06-20,1


In [203]:
def fill_in_missing_dates(df, date_col_name = 'date',date_order = 'asc', fill_value = 0):
    idx = pd.date_range(min(df[date_col_name]), max(df[date_col_name]), freq = "D")
    df.set_index(date_col_name,drop=True,inplace=True)
    df.index = pd.DatetimeIndex(df.index)    
    df = df.reindex(idx,fill_value=fill_value)
    df[date_col_name] = pd.DatetimeIndex(df.index)
    return df

df_kiwo_new = fill_in_missing_dates(df_kiwo, date_col_name = 'Datum', fill_value = 0)

In [204]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_kiwo_new['Datum'], y=df_kiwo_new['KielerWoche'], name="Category 1"))

fig.update_layout(title_text='Kieler Woche',
                  xaxis_rangeslider_visible=True)
fig.show()

### Sales

In [98]:
df_sales_raw = pd.read_csv('../data/umsatzdaten_gekuerzt.csv')#, header=0, parse_dates=[0], index_col=0, squeeze=True)
df_sales_raw.head()
#print(type(df_sales))

,Datum,Warengruppe,Umsatz
0,2013-07-01,1,148.828353
1,2013-07-02,1,159.793757
2,2013-07-03,1,111.885594
3,2013-07-04,1,168.864941
4,2013-07-05,1,171.280754


In [105]:
def get_category_from_sales(df, category):
    # get values from data frame with specific category ("Warengruppe")
    df_category = df[df.Warengruppe == category]
    
    # delete column
    df_category.drop("Warengruppe", axis=1, inplace=True)
    
    # transfer to series type
    df_category.set_index('Datum', inplace=True)
    
    # rename column name
    df_category.rename(columns=lambda x: x + '_' + str(category), inplace=True)
    
    return df_category

In [111]:
# 
ts_sales_1 = get_category_from_sales(df_sales_raw, 1)
ts_sales_2 = get_category_from_sales(df_sales_raw, 2)
ts_sales_3 = get_category_from_sales(df_sales_raw, 3)
ts_sales_4 = get_category_from_sales(df_sales_raw, 4)
ts_sales_5 = get_category_from_sales(df_sales_raw, 5)

ts_sales_2.head()

,Umsatz_2
Datum,
2013-07-01,535.856285
2013-07-02,546.780787
2013-07-03,427.343259
2013-07-04,454.859641
2013-07-05,492.818804


In [143]:
from functools import reduce

df_list = [ts_sales_1, ts_sales_2, ts_sales_3, ts_sales_4, ts_sales_5]

result = reduce(lambda left, right: pd.merge(left, right, left_on='Datum', right_on='Datum', how='outer'), df_list)
result.head()

,Umsatz_1,Umsatz_2,Umsatz_3,Umsatz_4,Umsatz_5
Datum,,,,,
2013-07-01,148.828353,535.856285,201.198426,65.890169,317.475875
2013-07-02,159.793757,546.780787,265.261254,74.543917,383.628682
2013-07-03,111.885594,427.343259,210.260241,69.262728,305.523072
2013-07-04,168.864941,454.859641,190.686641,61.490175,308.408168
2013-07-05,171.280754,492.818804,181.644870,86.759861,355.518770


In [144]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts_sales.index, y=ts_sales['Umsatz_1'], name="Category 1"))

fig.add_trace(go.Scatter(x=ts_sales.index, y=ts_sales['Umsatz_2'], name="Category 2"))

fig.add_trace(go.Scatter(x=ts_sales.index, y=ts_sales['Umsatz_3'], name="Category 3"))

fig.update_layout(title_text='Sales with Category',
                  xaxis_rangeslider_visible=True)
fig.show()

### Weather

In [137]:
df_weather = pd.read_csv('../data/wetter.csv')#, header=0, parse_dates=[0], index_col=0, squeeze=True)
#df_weather.set_index('Datum', inplace=True)
df_weather.head()

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode
0,2012-01-01,8.0,9.8250,14,58.0
1,2012-01-02,7.0,7.4375,12,NaN
2,2012-01-03,8.0,5.5375,18,63.0
3,2012-01-04,4.0,5.6875,19,80.0
4,2012-01-05,6.0,5.3000,23,80.0


In [138]:

print(type(df_weather))

<class 'pandas.core.frame.DataFrame'>


In [139]:
df_weather_split = df_weather[df_weather.Datum > '2013-05-01']
df_weather_split.head()

,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode
335,2013-05-02,1.0,11.0375,12,NaN
336,2013-05-03,0.0,12.9125,9,NaN
337,2013-05-04,0.0,17.9500,18,NaN
338,2013-05-05,3.0,16.2125,10,NaN
339,2013-05-06,0.0,15.6750,8,NaN


In [140]:
df_weather_split.set_index('Datum', inplace=True)
print(type(df_weather_split))
df_weather_split.head()

<class 'pandas.core.frame.DataFrame'>


,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode
Datum,,,,
2013-05-02,1.0,11.0375,12,NaN
2013-05-03,0.0,12.9125,9,NaN
2013-05-04,0.0,17.9500,18,NaN
2013-05-05,3.0,16.2125,10,NaN
2013-05-06,0.0,15.6750,8,NaN


In [141]:
print(type(df_weather_split))

<class 'pandas.core.frame.DataFrame'>


In [133]:
df = df_weather_split

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Datum'], y=df['Temperatur'], name="AAPL High",
                         line_color='deepskyblue'))


fig.update_layout(title_text='Time Series with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()